In [1]:
# Stop warnings
# -------------
import warnings
warnings.filterwarnings("ignore")

# General imports
# ---------------
import os
import sys
import json
import glob
import numpy as np
import h5py
import pandas as pd
opj = os.path.join

# Get inputs
# ----------
subject = 'sub-001'
task = 'GazeCenterFS'
preproc = 'fmriprep_dct_pca'

# Define analysis parameters
with open('mri_analysis/settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

In [2]:
def weighted_regression(x_reg,y_reg,weight_reg):
    """
    Function to compute regression parameter weighted by a matrix (e.g. r2 value).

    Parameters
    ----------
    x_reg : array (1D)
        x values to regress
    y_reg : array
        y values to regress
    weight_reg : array (1D) 
        weight values (0 to 1) for weighted regression

    Returns
    -------
    coef_reg : array
        regression coefficient
    intercept_reg : str
        regression intercept
    """

    from sklearn import linear_model
    regr = linear_model.LinearRegression()
    
    def m(x, w):
        return np.sum(x * w) / np.sum(w)

    def cov(x, y, w):
        # see https://www2.microstrategy.com/producthelp/archive/10.8/FunctionsRef/Content/FuncRef/WeightedCov__weighted_covariance_.htm
        return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

    def weighted_corr(x, y, w):
        # see https://www2.microstrategy.com/producthelp/10.4/FunctionsRef/Content/FuncRef/WeightedCorr__weighted_correlation_.htm
        return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

    x_reg_nan = x_reg[(~np.isnan(x_reg) & ~np.isnan(y_reg))]
    y_reg_nan = y_reg[(~np.isnan(x_reg) & ~np.isnan(y_reg))]
    weight_reg_nan = weight_reg[~np.isnan(weight_reg)]

    regr.fit(x_reg_nan.reshape(-1, 1), y_reg_nan.reshape(-1, 1),weight_reg_nan)
    coef_reg, intercept_reg = regr.coef_, regr.intercept_

    return coef_reg, intercept_reg

In [3]:
# Define folders and settings
# ---------------------------
base_dir = analysis_info['base_dir']
h5_dir = "{base_dir}/pp_data/{subject}/gauss/h5".format(base_dir = base_dir, subject = subject)

# load deriv data
rsq_idx, ecc_idx, size_idx, x_idx, y_idx, hemi_idx = 0, 1, 4, 8, 9, 10
rsq_dict, ecc_dict, size_dict, x_dict, y_dict, hemi_dict = {}, {}, {}, {}, {}, {}

for roi in analysis_info['rois']:
    
    h5_file = h5py.File("{h5_dir}/{roi}_{task}_{preproc}.h5".format(h5_dir = h5_dir, roi = roi, task = task, preproc = preproc),'r')
    deriv_data = h5_file['{folder_alias}/derivatives'.format(folder_alias = 'pRF')]

    rsq_dict.update({roi:deriv_data[:,rsq_idx]})
    ecc_dict.update({roi:deriv_data[:,ecc_idx]})
    size_dict.update({roi:deriv_data[:,size_idx]})
    x_dict.update({roi:deriv_data[:,x_idx]})
    y_dict.update({roi:deriv_data[:,y_idx]})
    hemi_dict.update({roi:deriv_data[:,hemi_idx]})

# make a dataframe with different array size
df_rsq = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in rsq_dict.items()]))
df_ecc = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in ecc_dict.items()]))
df_size = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in size_dict.items()]))
df_x = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in x_dict.items()]))
df_y = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in y_dict.items()]))
df_hemi = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in hemi_dict.items()]))

In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",          # figure axes color
                        axes_width=2,                           # figureaxes line width
                        axes_font_size=15,                      # font size of axes
                        bg_col="rgba(255, 255, 255, 1)",        # figure background color
                        font='Helvetica',                       # general font used
                        title_font_size=30,                     # font size of titles
                        )

fig_template=go.layout.Template()

# Violin plots
fig_template.data.violin = [go.Violin(
                                box_visible=False,
                                points=False,
                                opacity=1,
                                line_color= "rgba(0, 0, 0, 1)",
                                line_width=2,
                                width=0.8,
                                marker_symbol='x',
                                marker_opacity=0.5,
                                hoveron='violins',
                                meanline_visible=True,
                                meanline_color="rgba(0, 0, 0, 1)",
                                meanline_width=2,
                                showlegend=False,
                                )]
# Pie plots
fig_template.data.pie = [go.Pie(showlegend=False,
                                textposition=["inside","none"],
                                marker_line_color=['rgba(0,0,00,1)','rgba(255,255,255,0)'],
                                marker_line_width=[2,0],
                                rotation=0,
                                direction="clockwise",
                                hole=0.4,
                                sort=False,
                                )]
# Layout
fig_template.layout = (go.Layout(# general
                                font_family=template_specs['font'],
                                font_size=template_specs['axes_font_size'],
                                plot_bgcolor=template_specs['bg_col'],

                                # x axis
                                xaxis_visible=True,
                                xaxis_linewidth=template_specs['axes_width'],
                                xaxis_color= template_specs['axes_color'],
                                xaxis_showgrid=False,
                                xaxis_ticks="outside",
                                xaxis_ticklen=0,
                                xaxis_tickwidth = template_specs['axes_width'],
                                xaxis_tickfont_family=template_specs['font'],
                                xaxis_tickfont_size=template_specs['axes_font_size'],
                                xaxis_zeroline=False,
                                xaxis_zerolinecolor=template_specs['axes_color'],
                                xaxis_zerolinewidth=template_specs['axes_width'],
                                xaxis_range=[0,1],
                                
                                # # y axis
                                yaxis_visible=False,
                                yaxis_linewidth=0,
                                yaxis_color= template_specs['axes_color'],
                                yaxis_showgrid=False,
                                yaxis_ticks="outside",
                                yaxis_ticklen=0,
                                yaxis_tickwidth = template_specs['axes_width'],
                                yaxis_tickfont_family=template_specs['font'],
                                yaxis_tickfont_size=template_specs['axes_font_size'],
                                yaxis_zeroline=False,
                                yaxis_zerolinecolor=template_specs['axes_color'],
                                yaxis_zerolinewidth=template_specs['axes_width'],
                                
                                ))

# Annotations
fig_template.layout.annotationdefaults = go.layout.Annotation(
                                font_family=template_specs['font'],
                                font_size=template_specs['axes_font_size'])


# to do
2. compute relation eccentricity size relationship 
    - add filter of data
    - add ecc/size dots or subselection of it
4. make plot with two entries (condition 1 and 2 => split violoin, make two circle, make 2 sets of lines changing hue)
5. make entries slider to pick the condition to plot
6. save as pdf and html
3. fix hover (https://plotly.com/python/hover-text-and-formatting/)
6. attach axes together
7. add title to overall plots
7. take care of legend (in the figure, different for each, include area and line)

In [28]:
def rgb2rgba(input_col,alpha_val):
    rgba_col = "rgba{}, {})".format(input_col[3:-1],alpha_val)
    return rgba_col

fig_height, fig_width = 900,1200
rois_colors = px.colors.qualitative.Prism
rois_colors.append('rgb(200, 200, 200)')
rois = analysis_info['rois']
rows, cols = 3, 12

column_widths = [1,1,1,1,1,1,1,1,1,1,1,1,]
row_heights = [4,1,4]

sb_specs = [[{},{},{},{},{},{},{},{},{},{},{},{}],
            [{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'},
             {'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'}],
            [{'colspan':4},None,None,None,{'colspan':4},None,None,None,{'colspan':4},None,None,None]]

fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.04, horizontal_spacing=0.02,
                    column_widths=column_widths, row_heights=row_heights)

cols_violin, rows_violin = [1,2,3,4,5,6,7,8,9,10,11,12], [1,1,1,1,1,1,1,1,1,1,1,1]
cols_pie, rows_pie       = [1,2,3,4,5,6,7,8,9,10,11,12], [2,2,2,2,2,2,2,2,2,2,2,2]
cols_trace, rows_trace   = [1,1,1,1,5,5,5,5,9, 9, 9, 9], [3,3,3,3,3,3,3,3,3,3,3,3]

# Ecentricity/size relationship
y_label_trace, x_label_trace, trace_range = 'pRF size (dva)', 'pRF eccentricity (dva)', [0,15]
line_x = np.linspace(trace_range[0], trace_range[1], 60)

for num,(roi,roi_color) in enumerate(zip(rois,rois_colors)):
    
    # r2 violin plots
    fig.append_trace(go.Violin(y=df_rsq[roi], name=roi, span=[0, 1], orientation= "v", spanmode='manual', fillcolor=roi_color), row=rows_violin[num], col=cols_violin[num])

    # contra-laterality ratio
    cl_ratio = np.mean([np.sum(df_rsq[df_x[df_hemi==2]<0][roi])/np.sum(df_rsq[df_hemi==2][roi]),
                        np.sum(df_rsq[df_x[df_hemi==1]>0][roi])/np.sum(df_rsq[df_hemi==1][roi])])

    fig.append_trace(go.Pie(labels=["Contra-lateral","Ipsi-lateral"],
                            values=[cl_ratio,1-cl_ratio], marker_colors=[roi_color,'rgba(255,255,255,0)']),row=rows_pie[num], col=cols_pie[num])

    # eccentricity size regression
    ecc_size_coeff, ecc_size_intercept = weighted_regression(np.array(df_ecc[roi]),np.array(df_size[roi]),np.array(df_rsq[roi]))
    line_y = ecc_size_coeff*line_x+ecc_size_intercept
    # eb, eb_low, eb_high = 2, line_y-eb/2, line_y+eb/2
    # fig.append_trace(go.Scatter(x=np.hstack((line_x,line_x[::-1])), y=np.hstack((eb_low,eb_high[::-1])), mode='lines',
    #                             fill='toself',fillcolor=rgb2rgba(roi_color,0.2),line_color='rgba(0,0,0,0)', showlegend=False),row=rows_trace[num], col=cols_trace[num])
    fig.append_trace(go.Scatter(x=line_x, y=line_y[0], name = roi, mode='lines', line_width=2, line_color=roi_color,showlegend=False),row=rows_trace[num], col=cols_trace[num])

fig.add_annotation(xref="paper", yref="paper", x=-0.08, y=0.5, text='Contra-<br>laterality ratio', showarrow=False, textangle=-90,font_family=template_specs['font'],
                                font_size=17)

fig.layout.update(  # figure settings
                    template=fig_template, width=fig_width, height=fig_height, margin_l=100, margin_r=20, margin_t=50, margin_b=100,
                    # range violin
                    yaxis_range=[0,1], yaxis2_range=[0,1],yaxis3_range=[0,1],yaxis4_range=[0,1], yaxis5_range=[0,1], yaxis6_range=[0,1],
                    yaxis7_range=[0,1],yaxis8_range=[0,1],yaxis9_range=[0,1],yaxis10_range=[0,1],yaxis11_range=[0,1],yaxis12_range=[0,1],
                    # y axis violin
                    yaxis_visible=True, yaxis_linewidth=template_specs['axes_width'], yaxis_title_text='R\u00b2', yaxis_ticklen=8, 
                    # traces #13
                    yaxis13_visible=True, yaxis13_linewidth=template_specs['axes_width'], yaxis13_title_text=y_label_trace, yaxis13_range=trace_range, yaxis13_ticklen=8,
                    xaxis13_visible=True, xaxis13_linewidth=template_specs['axes_width'], xaxis13_title_text=x_label_trace, xaxis13_range=trace_range, xaxis13_ticklen=8,
                    # traces #14
                    yaxis14_visible=True, yaxis14_linewidth=template_specs['axes_width'], yaxis14_showticklabels=False, yaxis14_range=trace_range, yaxis14_ticklen=8,
                    xaxis14_visible=True, xaxis14_linewidth=template_specs['axes_width'], xaxis14_title_text=x_label_trace, xaxis14_range=trace_range, xaxis14_ticklen=8,
                    # traces #15
                    yaxis15_visible=True, yaxis15_linewidth=template_specs['axes_width'], yaxis15_showticklabels=False, yaxis15_range=trace_range, yaxis15_ticklen=8,
                    xaxis15_visible=True, xaxis15_linewidth=template_specs['axes_width'], xaxis15_title_text=x_label_trace, xaxis15_range=trace_range, xaxis15_ticklen=8,
                  )

fig.show()

